In [1]:
!pip install geopandas shapely tqdm

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np


In [7]:
# Load shp file
gdf_origin = gpd.read_file("T:/California/Sacramento/Sa_pop.shp")
gdf_dest = gpd.read_file("T:/California/Sacramento/Sa_vac.shp")

In [8]:
# Convert CRS to UTM projected coordinates (select according to region)
gdf_origin = gdf_origin.to_crs("EPSG:3857")
gdf_dest = gdf_dest.to_crs("EPSG:3857")

In [9]:
# 对于 gdf_origin GeoDataFrame
origin_points = gdf_origin.geometry.apply(lambda geom: (geom.x, geom.y) if geom is not None else (None, None)).tolist()

# 对于 gdf_dest GeoDataFrame
dest_points = gdf_dest.geometry.apply(lambda geom: (geom.x, geom.y) if geom is not None else (None, None)).tolist()

In [12]:
from tqdm import tqdm
from shapely.geometry import Point
from scipy.spatial import cKDTree
import geopandas as gpd
import numpy as np
import os

k = len(gdf_dest)
dest_coords = np.array(list(zip(gdf_dest.geometry.x, gdf_dest.geometry.y)))
tree = cKDTree(dest_coords)
#这里要设定用几个excel跑，如果文件很大，就多用几个，上面有orgin和dest的输出，可以大致估计一下
num_splits = 1
split_size = len(gdf_origin) // num_splits
splits = np.array_split(gdf_origin, num_splits)

for split_index, split in enumerate(splits):
    split_origin_coords = np.array([(row.geometry.x, row.geometry.y) for index, row in split.iterrows()])
    distances, indices = tree.query(split_origin_coords, k=k)
    data = []

    for i in tqdm(range(len(split_origin_coords)), desc=f"Processing split {split_index+1}/{num_splits}"):
        origin = split.iloc[i]
        origin_point = Point(split_origin_coords[i])
        for j in range(k):
            dest_idx = indices[i][j]
            dest_point = Point(dest_coords[dest_idx])
            distance = distances[i][j] / 1000  # Convert distance to kilometers
            data.append({
                'origin': origin_point,
                'dest': dest_point,
                'distance': distance,
                'pointid': origin.get('pointid', None),
                'grid_code': origin.get('grid_code', None)
            })

    gdf_result = gpd.GeoDataFrame(data, geometry='dest')
    gdf_result.set_crs(gdf_dest.crs, inplace=True)
    #改地址
    output_dir = os.path.dirname("T:/California/")
    os.makedirs(output_dir, exist_ok=True)
    #改名
    file_path = os.path.join(output_dir, f"Sa_pop_{split_index+1}.csv")
    gdf_result.to_csv(file_path, index=False)

C:\Users\u3603957\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
C:\Users\u3603957\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'GeoDataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'GeoDataFrame.transpose' instead.
  return bound(*args, **kwds)
Processing split 1/1: 100%|████████████████████████████████████████████████████████| 1812/1812 [01:49<00:00, 16.51it/s]


In [1]:


import pandas as pd
from tqdm.auto import tqdm
gdf_result = pd.read_csv("C:/Users/u3603957/Downloads/Uganda/Uganda_pop_4.csv")
gdf_result['origin'] = gdf_result['origin'].astype(str)

def filter_rows(group):
    # 判断组内所有distance是否都大于100
    if (group['distance'] > 100).all():
        # 再判断组内所有distance是否都大于50
        if (group['distance'] > 120).all():
            # 如果是，则保留最小的一个distance
            return group.nsmallest(1, 'distance')
        else:
            # 否则，保留所有小于等于50的distance，并从这些中保留最小的三个
            return group[group['distance'] <= 120].nsmallest(3, 'distance')
    else:
        # 否则，保留所有小于等于30的distance
        return group[group['distance'] <= 100]

tqdm.pandas(desc="Processing groups")
result_new = gdf_result.groupby('origin', as_index=False, group_keys=False).progress_apply(filter_rows)



import geopandas as gpd
from shapely.wkt import loads
if isinstance(result_new['origin'].iloc[0], str):
    result_new['origin'] = result_new['origin'].apply(loads)
if isinstance(result_new['dest'].iloc[0], str):
    result_new['dest'] = result_new['dest'].apply(loads)

gdf_result = gpd.GeoDataFrame(result_new, geometry='origin', crs="EPSG:3857")
gdf_result.set_geometry('origin', inplace=True)
gdf_result['origin'] = gdf_result['origin'].to_crs("EPSG:4326")

gdf_result['dest'] = gpd.GeoSeries(gdf_result['dest'], crs="EPSG:3857")
gdf_result['dest'] = gdf_result['dest'].to_crs("EPSG:4326")

gdf_result.to_csv("C:/Users/u3603957/Downloads/Uganda/Uganda_filter_4.csv")


Processing groups:   0%|          | 0/26912 [00:00<?, ?it/s]